In [11]:
%cd ../..

/home/philippe


/home/philippe/.conda/envs/mol-distill/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [12]:
import os.path

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from molDistill.utils.notebooks import *

MODELS_TO_EVAL = [
    "ChemBertMLM-10M",
    "ChemBertMTR-77M",
    "ChemGPT-1.2B",
    "GraphMVP",
    "GROVER",
    "GraphLog",
    "GraphCL",
    "InfoGraph",
    "FRAD_QM9",
    "MolR_gat",
    "ThreeDInfomax",
    "custom:MOSES_512_10_lr1e-4_gine"
]

DATASETS = df_metadata[df_metadata.task_type == "cls"].index.tolist()

# Mean Performances

In [13]:
df_base= get_all_results(MODELS_TO_EVAL, "downstream_results", DATASETS)

df, order = aggregate_results_with_ci(df_base)
style,latex = style_df_ci(df, order)

style

FileNotFoundError: [Errno 2] No such file or directory: 'downstream_results/ChemBertMLM-10M'

In [ ]:
table_path = "/home/philippe/Distill/latex/Distillation-MI-ICLR/tables/molecules/molecule_results_cls.tex"
with open(table_path, "w") as f:
    f.write(latex)

# Rankings

In [ ]:
df_base= get_all_results(MODELS_TO_EVAL, "downstream_results", DATASETS)
df_base.reset_index(inplace=True, drop=True)

step = df_base.embedder.value_counts().max()
df_base["id"] = df_base.index%step

df_base

In [ ]:
ranked_df = get_ranked_df(df_base)

ranked_df.columns = ["embedder"] + [df_metadata.loc[c, "short_name"] for c in ranked_df.columns[1:]]

melted_ranked = ranked_df.melt(id_vars="embedder", var_name="dataset", value_name="meanrank")
order_datas =ranked_df.set_index("embedder").transpose()["student"].sort_values().index


#define colors for models so that student has more saturation
colors = sns.color_palette("husl", len(MODELS_TO_EVAL), desat=0.4)
colors[-1] = sns.color_palette("husl", len(MODELS_TO_EVAL))[-1]

cmapping = {model.replace(
    "custom:MOSES_512_10_lr1e-4_gine", "student"
): color for model, color in zip(MODELS_TO_EVAL, colors)}

fig,ax = plt.subplots(figsize=(20,3))

sns.barplot(data=melted_ranked, x="dataset", y="meanrank", hue="embedder", ax=ax, order=order_datas, palette=cmapping)
sns.despine()
#rotate x labels
plt.xticks(rotation=45)
#Move legend outside
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=1)

In [ ]:
style = get_ranked_df(df_base, order)
style["embedder"] = style["embedder"].str.replace("_", " ")

style.set_index("embedder", inplace=True)

style.columns= pd.MultiIndex.from_tuples([(
    df_metadata.loc[c, "category"], df_metadata.loc[c, "short_name"]) for c in style.columns])
style = style[sorted(style.columns, key=lambda x: x[0])]

style = style.style.highlight_min(axis=0, props='bfseries:')

style


In [ ]:
col_format = "r|"
over_cols = None
for ov_col, col in style.columns:
    if over_cols != ov_col:
        col_format += "|"
        over_cols = ov_col
    col_format += "c"
col_format += "|"


latex = style.to_latex(
    column_format=col_format,
    multicol_align="|c|",
    siunitx=True,
)


table_path = "/home/philippe/Distill/latex/Distillation-MI-ICLR/tables/molecules/molecule_results_cls_rank.tex"
with open(table_path, "w") as f:
    f.write(latex)